In [1]:
# import necessary stuff and python-wrapper of verse
import os
import pprint
import numpy as np
import pandas as pd
import networkx as nx
import tensorflow as tf
import pickle
from tensorflow.contrib.tensorboard.plugins import projector

from verse.python.wrapper import VERSE
from multi_class_classification import MultiClassClassification
from multi_label_classification import MultiLabelClassification
from clustering import Clustering
from link_prediction import LinkPrediction
from experiment import Experiment

/home/julius/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# initialize pretty printer
pp = pprint.PrettyPrinter(indent=4, depth=8)

In [3]:
# configure telegram notifier bot
my_telegram_config = {
    "telegram": {
        "token": "350553078:AAEu70JDqMFcG_x5eBD3nqccTvc4aFNMKkg",
        "chat_id": "126551968",
        "verbose": 1
    }
}

In [4]:
# define hyper-parameters
n_hidden = 128

In [5]:
# read *.bin file with precomputed embeddings
embeddings_file_path = 'results/blogcatalog/blogcatalog_verse_embeddings.bin'
embeddings_file = open(embeddings_file_path, "r")
embeddings_file_content = np.fromfile(embeddings_file, dtype=np.float32)
num_of_nodes = int(np.shape(embeddings_file_content)[0] / n_hidden)
verse_ppr_embeddings = embeddings_file_content.reshape((num_of_nodes, n_hidden))

In [6]:
# check shape of trained embedding matrix
np.shape(verse_ppr_embeddings)

(10351, 128)

In [7]:
# define dataset file paths
dataset_path = 'data/BlogCatalog-dataset/data/'
friend_edges_csv_path = dataset_path + 'edges.csv'
group_edges_csv_path = dataset_path + 'group-edges.csv'
groups_csv_path = dataset_path + 'groups.csv'
bloggers_csv_path = dataset_path + 'nodes.csv'

In [8]:
# store cvs contents in dataframe
friend_edges_df = pd.read_csv(friend_edges_csv_path, sep=',', header=None, dtype={0: str, 1:str})
group_edges_df = pd.read_csv(group_edges_csv_path, sep=',', header=None, dtype={0: str, 1:str})
groups_df = pd.read_csv(groups_csv_path, sep=',', header=None, dtype={0: str})
bloggers_df = pd.read_csv(bloggers_csv_path, sep=',', header=None, dtype={0: str})

In [9]:
# give bloggers and groups unique node-ids
bloggers_df[0] = 'b' + bloggers_df[0]
friend_edges_df = 'b' + friend_edges_df
groups_df[0] = 'g' + groups_df[0]
group_edges_df[0] = 'b' + group_edges_df[0]
group_edges_df[1] = 'g' + group_edges_df[1]

In [10]:
# define networkx graph
blog_catalog_graph = nx.Graph()

In [11]:
# define node and edge label constants
IS_MEMBER_OF = 'is_member_of'
IS_FRIEND_WITH = 'is_friend_with'
BLOGGER = 'blogger'
GROUP = 'group'

In [12]:
# add blogger nodes to graph
blog_catalog_graph.add_nodes_from(bloggers_df[0].tolist(), label=BLOGGER)
print("{} nodes in graph".format(blog_catalog_graph.number_of_nodes()))
blog_catalog_graph.add_nodes_from(groups_df[0].tolist(), label=GROUP)
print("{} nodes in graph".format(blog_catalog_graph.number_of_nodes()))

10312 nodes in graph
10351 nodes in graph


In [13]:
# create edge tuples from dataframe
group_edges = list(zip(group_edges_df[0].tolist(), group_edges_df[1].tolist()))
friend_edges = list(zip(friend_edges_df[0].tolist(), friend_edges_df[1].tolist()))

In [14]:
# add (blogger)-[is_member_of]-(group) edges to graph
blog_catalog_graph.add_edges_from(group_edges, label=IS_MEMBER_OF)
print("{} edges in graph".format(blog_catalog_graph.number_of_edges()))
print("{} nodes in graph".format(blog_catalog_graph.number_of_nodes()))

14476 edges in graph
10351 nodes in graph


In [15]:
# add (blogger)-[is_friend_with]-(blogger) edges to graph
blog_catalog_graph.add_edges_from(friend_edges, label=IS_FRIEND_WITH)
print("{} edges in graph".format(blog_catalog_graph.number_of_edges()))
print("{} nodes in graph".format(blog_catalog_graph.number_of_nodes()))

348459 edges in graph
10351 nodes in graph


In [16]:
# compute average degree of all nodes in graph
node_degrees = np.array(list(dict(blog_catalog_graph.degree(list(blog_catalog_graph.nodes))).values()),dtype=np.int64)
avg_node_degree = np.mean(node_degrees)
print("The avg. node degree is {}".format(np.round(avg_node_degree, decimals=2)))

The avg. node degree is 67.33


In [17]:
# load id-to-node mapping of verse embeddings
id2node_filepath = 'data/BlogCatalog-dataset/data/blogcatalog_mapping_ids_to_nodes.p'
id_2_node = {}
with open(id2node_filepath, 'rb') as id_2_node_file:
    id_2_node = pickle.load(id_2_node_file)

In [18]:
# define tensorflow log-directory
LOG_DIR = os.path.dirname(os.path.realpath('results/blogcatalog/visualizations/')) + '/visualizations/'

In [19]:
# define node labels
BLOGGER = 'blogger'
GROUP = 'group'

In [20]:
# write node labels to metadata tensorflow file
node_labels = [blog_catalog_graph.nodes[id_2_node[i]]['label'] for i in range(np.shape(verse_ppr_embeddings)[0])]   

In [21]:
# write node label list to file
EXPORT_NODE_LABEL_META_DATA = True
metadata_file_path = LOG_DIR + 'metadata_verse_node_labels.csv'

if EXPORT_NODE_LABEL_META_DATA:
    metadata_file = open(metadata_file_path, 'w')

    for node_label in node_labels:
        metadata_file.write("{}\n".format(node_label))

    metadata_file.close()

In [22]:
# define tensorflow projector variables
embedding_var = tf.Variable(verse_ppr_embeddings, name='blogcatalog_verse_embeddings')

In [23]:
# train tensorflow projector
with tf.Session() as sess:
    sess.run(embedding_var.initializer)
    
    writer = tf.summary.FileWriter(LOG_DIR, sess.graph)
    
    config = projector.ProjectorConfig()
    embedding = config.embeddings.add()
    embedding.tensor_name = embedding_var.name
    embedding.metadata_path = metadata_file_path
    
    projector.visualize_embeddings(writer, config)
    
    saver_embed = tf.train.Saver([embedding_var])
    saver_embed.save(sess, LOG_DIR + 'blogcatalog_verse_embeddings_viz.ckpt', 1)